# Mortgage Workflow with Deep Learning

## Dataset

The dataset used with this workflow is derived from [Fannie Mae’s Single-Family Loan Performance Data](http://www.fanniemae.com/portal/funding-the-market/data/loan-performance-data.html) with all rights reserved by Fannie Mae. This processed dataset is redistributed with permission and consent from Fannie Mae.

Preprocessing ETL has already been precalculated and is located at /tmp/eoldridge/fnma_full_data_proc_out4/dnn/

## PyTorch Deep Neural Network

### Model
The model constructed below starts with an initial embedding layer ([`torch.nn.EmbeddingBag`](https://pytorch.org/docs/stable/nn.html#embeddingbag)) that takes the indices from the ETL pipeline, looks up the embeddings in the hash table and takes their mean. This vector then passes to a [multilayer perceptron](https://en.wikipedia.org/wiki/Multilayer_perceptron) which finally outputs a single score.

Many of the model architecture parameters can be configured by the user such as embedding dimension, number and size of hidden layers, and activation functions.

### Training
To cut down on boilerplate code and realize the benefits of [early stopping](https://en.wikipedia.org/wiki/Early_stopping)
we use the [`ignite`](https://pytorch.org/ignite/) library.


## Requirements
Beyond the dependencies that come installed in the standard 
[RAPIDS docker containers](https://hub.docker.com/r/rapidsai/rapidsai) we'll also
need the following `pip` dependencies installed:

In [1]:
!pip install torch pytorch-ignite

     |################################| 676.9MB 20kB/s 
     |################################| 81kB 34.7MB/s 


In [2]:
!pip install snakeviz

     |################################| 286kB 9.4MB/s 


## CODE
Most of the details are buried/organized within the .py files.

### Imports

In [1]:
from collections import defaultdict, OrderedDict
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
import pyarrow.parquet as pq

In [2]:
import cudf
cudf.__version__

'0.7.2+0.g3ebd286.dirty'

In [3]:
import pdb

In [4]:
%load_ext autoreload
%autoreload 2

## Configuration

#### ETL - Discretization

In [5]:
max_quantiles = 20  # Used for computing histograms of continuous features
num_features = 2 ** 22  # When hashing features range will be [0, num_features)

#### Training - Model Details

In [6]:
embedding_size = 64
hidden_dims = [600,600,600,600]

device = 'cuda'
dropout = None  # Can add dropout probability in [0, 1] here
activation = nn.ReLU()

batch_size = 80960

## Torch Dataset from Parquet
The preprocessing ETL has already been precalculated and is stored at: /tmp/eoldridge/fnma_full_data_proc_out4/dnn/

In [7]:
data_dir = '/data/mortgage/'
!ls -al --block-size=M /data/mortgage/

total 1M
drwxr-xr-x 1 10128 10004 0M May 29 18:38 .
drwxr-xr-x 3 root  root  1M Jul  9 14:14 ..
drwxr-xr-x 1 10128 10004 0M May 29 18:38 test
drwxr-xr-x 1 10128 10004 0M May 29 18:38 train
drwxr-xr-x 1 10128 10004 0M May 29 18:38 validation


### Training starts here

In [8]:
from training import run_training
from model import MortgageNetwork

In [9]:
model = None
model = MortgageNetwork(num_features, embedding_size, hidden_dims,
                        dropout=dropout, activation=activation, use_cuda=True)

In [10]:
model.device

device(type='cuda')

In [11]:
%load_ext snakeviz

In [12]:
%snakeviz run_training(model, data_dir, batch_size=batch_size, batch_dataload=True, num_workers=8)

Epoch[1] Iteration[6/107] Loss: 0.22248 Example/s: 34978.405 (Total examples: 485760)
Epoch[1] Iteration[12/107] Loss: 0.12772 Example/s: 63785.024 (Total examples: 971520)
Epoch[1] Iteration[18/107] Loss: 0.11654 Example/s: 88085.466 (Total examples: 1457280)
Epoch[1] Iteration[24/107] Loss: 0.10037 Example/s: 108607.839 (Total examples: 1943040)
Epoch[1] Iteration[30/107] Loss: 0.09003 Example/s: 126223.986 (Total examples: 2428800)
Epoch[1] Iteration[36/107] Loss: 0.07810 Example/s: 141767.991 (Total examples: 2914560)
Epoch[1] Iteration[42/107] Loss: 0.06543 Example/s: 155476.671 (Total examples: 3400320)
Epoch[1] Iteration[48/107] Loss: 0.06171 Example/s: 167460.702 (Total examples: 3886080)
Epoch[1] Iteration[54/107] Loss: 0.06012 Example/s: 178209.619 (Total examples: 4371840)
Epoch[1] Iteration[60/107] Loss: 0.05507 Example/s: 187614.853 (Total examples: 4857600)
Epoch[1] Iteration[66/107] Loss: 0.05189 Example/s: 196138.810 (Total examples: 5343360)
Epoch[1] Iteration[72/107] 